| <h1><b>Pengenalan Plotting</b></h1> |  ![](files/piksel-logo-small.png) |
|:---------|----------:|

* **Produk yang Digunakan:** 
s2_l2a

* **Prasyarat:** Para pengguna notebook ini telah memahami:
    * Bagaimana menjalankan [Jupyter notebook](01_Jupyter_notebooks.ipynb)
    * Memeriksa ketersediaan [Piksel products and measurements](02_Product_dan_measurement.ipynb)
    * Cara [load data dari Piksel](03_Membuka_data.ipynb)

## 🔙 Pendahuluan
Visualisasi data merupakan bagian penting dalam pengolahan data Pengamatan Bumi (Earth Observation). Paket Python xarray menyediakan berbagai opsi plotting data yang sederhana, memungkinkan pengguna membuat plot dari dataset multidimensi dengan cepat. Untuk menghasilkan plot yang lebih kompleks dan informatif dari data Piksel, repositori Notebook Piksel menyediakan modul plotting khusus dengan fungsionalitas tambahan yang mudah digunakan.

Notebook ini memperkenalkan cara memvisualisasikan data satelit Piksel dari hasil kueri datacube. Notebook ini mencakup metode plotting xarray yang umum digunakan serta fungsi khusus dari deafrica_tools.plotting.

Topik yang dibahas meliputi:

 - Menampilkan area of interest (AOI) sebelum melakukan kueri datacube
 - Melakukan kueri datacube dan memuat data
 - Plotting data satu band (misalnya, satu band satelit)
    - Memilih dan plotting satu langkah waktu (timestep)
    - Plotting beberapa langkah waktu
    - Menyesuaikan tampilan plot
 - Plotting citra tiga band dalam warna asli (true color) atau palsu (false color)
    - Memetakan satu langkah waktu
    - Memetakan beberapa langkah waktu
    - Menyesuaikan tampilan plot

## 🏃‍♂️‍➡️ Memulai
Untuk menjalankan pengantar plotting data dari datacube, jalankan semua sel dalam notebook, mulai dari sel "Load packages".
Jika memerlukan bantuan dalam menjalankan sel notebook, silakan merujuk ke [Jupyter Notebooks notebook](01_Jupyter_notebooks.ipynb). 

### Memuat Library
Pertama, jalankan `%matplotlib inline`, untuk memastikan gambar ditampilkan dengan benar di Jupyter Notebook.

Selanjutnya, muat  `datacube` package, untuk mengambil data. 

Untuk memvisualisasikan data, impor fungsi pemetaan dari deafrica_tools.plotting. Dalam notebook ini, kita akan menggunakan `rgb` and `display_map`yang dapat diimpor dalam satu baris dengan memisahkan namanya dengan koma.

In [ ]:
%matplotlib inline

import datacube
from dea_tools.plotting import rgb, display_map

### Menghubungkan ke datacube
Kita kemudian terhubung ke basis data datacube agar dapat memuat data Piksel. 

In [ ]:
dc = datacube.Datacube(app="04_Plotting")

### Parameter analisis

Variabel berikut diperlukan untuk menetapkan kueri dalam notebook ini:
- `lat_range`: Rentang garis lintang yang akan dianalisis (misalnya, (-6.005237, -6.025759)). Untuk load waktu yang wajar, sebaiknya tetap dalam rentang ~0.1 derajat atau kurang.
- `lon_range`: Rentang garis bujur yang akan dianalisis (misalnya, (106.596046, 106.632157)). Untuk load waktu yang wajar, sebaiknya tetap dalam rentang ~0.1 derajat atau kurang.
- `time_range`: Rentang tanggal yang akan dianalisis (misalnya, ("2024-07-01", "2024-07-31")).


In [ ]:
lat_range = (-6.005237, -6.025759)
lon_range = (106.596046, 106.632157)
time_range = ("2024-07-01", "2024-07-31")

## 🗺️ Menampilkan lokasi
Sebelum menjalankan query serta mengekstrak dan menganalisis data, pastikan lokasi yang dipilih sudah benar.
Fungsi `display_map()` akan menampilkan area yang dipilih sebagai persegi panjang merah pada peta interaktif. Dengan mengklik titik mana pun di peta, kita dapat melihat koordinat garis lintang dan bujur dari titik tersebut.

In [ ]:
display_map(x=lon_range, y=lat_range)

## 💾 Menampilkan data
Variabel yang telah ditentukan sebelumnya digunakan di sini untuk melakukan kueri pada Piksel datacube menggunakan fungsi `dc.load()`dan memuat data yang diperkenalkan dalam Notebook [Membuka data](03_Membuka_data.ipynb).
Notebook ini menggunakan data Sentinel 2 `s2_l2a`.

In [ ]:
ds = dc.load(product="s2_l2a",
             measurements=['blue','green','red','nir','swir16','swir22'],
             x=lon_range,
             y=lat_range,
             time=time_range,
             output_crs='EPSG:32748',
             resolution=30
            )
print(ds)

## 🖨️ Memplot citra satu band

`xarray`menyediakan metode bawaan untuk memplot variabel atau measurement data secara individual. 
Misalnya, kita dapat membuat plot untuk satu measurement, seperti band satelit `swir_1` dari data yang telah dimuat sebelumnya.

Untuk melakukan ini, pertama-tama kita perlu mengakses band yang diinginkan sebagai `xarray.DataArray` (Untuk memahami perbedaan antara  `xarray.Dataset` dan `xarray.DataArray` silakan merujuk kembali ke [Membuka data](03_Membuka_data.ipynb) notebook:

In [ ]:
print(ds.swir16)

### Memilih dan memplot satu timestep.

Kita dapat melihat pada header objek bahwa `xarray.DataArray` ini memiliki data untuk enam timestep (misalnya. `<xarray.DataArray 'swir_1' (time: 6, y: 834, x: 644)>`).
Untuk memplot hanya satu timestep, kita perlu memilihnya menggunakan salah satu opsi berikut:

1. `.isel()`: .isel(): Ini singkatan dari "index selection" (pemilihan indeks) dan memungkinkan kita dengan mudah memilih timestep tertentu dari sebuah dataset dengan memberikan nomor observasi yang diinginkan. Dalam Python, perhitungan dimulai dari 0, jadi untuk memilih timestep pertama dalam xarray.DataArray, kita bisa menggunakan .isel(time=0).

In [ ]:
first_timestep = ds.swir16.isel(time=0)

print(first_timestep)

2. `.sel()`: Metode ini memungkinkan kita memilih data berdasarkan label koordinat dunia nyata, seperti `waktu`. Misalnya, dari bagian Koordinat, kita dapat memilih timestep pertama (yaitu, observasi pada 2 Juli 2024) dalam `xarray.DataArray` dengan menggunakan `.sel(time='2024-07-02')`.

In [ ]:
print(ds.time)

In [ ]:
first_timestep = ds.swir16.sel(time='2024-07-02')

print(first_timestep)

Sekarang kita dapat menggunakan metode .plot() untuk memvisualisasikan data swir1 pada timestep yang telah kita pilih:

In [ ]:
first_timestep.plot()

### Plotting Beberapa Timestep
Sering kali, berguna untuk membuat plot dari satu variabel pengukuran sepanjang waktu, misalnya untuk membandingkan perubahan antara observasi satelit atau summary dataset.
Untuk memplot beberapa citra sekaligus, kita dapat melewati langkah .isel() dan langsung memplot seluruh xarray.DataArray.
Agar setiap timestep ditampilkan dalam kolom terpisah dalam satu figur, kita dapat menggunakan:

In [ ]:
ds.swir16.plot(col="time")

> **Catatan**: Jenis plotting ini disebut "facetted plotting". Untuk informasi lebih lanjut, silahkan  merujuk ke [xarray documentation](http://xarray.pydata.org/en/stable/plotting.html#faceting)

### Kustomisasi tampilan plot
Anda mungkin menyadari bahwa plot di atas tampak gelap dan sulit dilihat dengan jelas. Untuk meningkatkan tampilan plot di xarray, kita dapat menggunakan argumen robust=True, yang akan mengoptimalkan warna dengan memangkas nilai ekstrem atau outlier.
Dengan menggunakan robust=True, batas warna akan dihitung berdasarkan persentil ke-2 dan ke-98 dari data, sehingga hasil visualisasi lebih jelas dan informatif.

In [ ]:
ds.swir16.plot(col="time", robust=True)

Saat memilih peta berwarna untuk sebuah plot, penting untuk menggunakan skema warna yang dapat dipahami secara logis oleh mata manusia. Peta berwarna terbaik adalah yang "perceptually uniform", yaitu peta berwarna yang berubah secara bertahap dari gelap ke terang, di mana setiap perubahan tingkat kecerahan sesuai dengan perubahan nilai data.
Beberapa contoh peta berwarna perceptually uniform yang direkomendasikan meliputi:

```
"viridis", "plasma", "inferno", "magma", "cividis"
```

> **Catatan**: Untuk membaca lebih lanjut tentang peta berwarna perceptually uniform dalam visualisasi data silahkan merujuk ke, [matplotlib documentation](https://matplotlib.org/stable/tutorials/colors/colormaps.html).

Penting juga untuk mempertimbangkan pengguna yang buta warna saat memilih peta berwarna.
xarray mendukung banyak peta berwarna dari kelompok [colorbrewer](https://colorbrewer2.org/#type=sequential&scheme=BuGn&n=3), yang telah dioptimalkan agar ramah bagi individu dengan buta warna.
Anda dapat menggunakan alat interaktif online untuk menjelajahi semua peta berwarna yang tersedia atau memilih salah satu dari opsi umum berikut:

```
"Greys", "Purples", "Blues", "Greens", "Oranges", "Reds",
"YlOrBr", "YlOrRd", "OrRd", "PuRd", "RdPu", "BuPu", 
"GnBu", "PuBu", "YlGnBu", "PuBuGn", "BuGn", "YlGn"
```

Untuk daftar lengkap peta berwarna yang tersedia, Anda dapat merujuk ke [daftar ini](https://matplotlib.org/stable/tutorials/colors/colormaps.html).

Sebagai contoh, untuk memvisualisasikan data menggunakan peta berwarna perceptually uniform magma, kita dapat menggunakan:


In [ ]:
ds.swir16.plot(col="time", robust=True, cmap="magma")

## 🖨️ Plotting Citra RGB True Color atau False Color

Meskipun xarray memudahkan pemetaan citra satu band, memvisualisasikan citra tiga band seperti foto berwarna tidak semudah itu.

Untuk menyederhanakan proses ini, repositori deafrica-sandbox-notebooks menyediakan fungsi kustom rgb(), yang dirancang khusus untuk menampilkan citra tiga band.
Fungsi rgb() memetakan tiga variabel data atau pengukuran dari dataset yang dimuat ke dalam saluran merah (Red), hijau (Green), dan biru (Blue) untuk menghasilkan citra warna.

•	Jika kita menggunakan band merah, hijau, dan biru, hasilnya adalah citra warna alami (true color), seperti yang terlihat oleh mata manusia.
•	Jika kita menggunakan band lain, seperti NIR, merah, dan hijau, hasilnya adalah citra warna semu (false color), yang sering digunakan dalam analisis satelit.
. Anda dapat belajar lebih lanjut tentang pewarnaan citra  [di sini](https://en.wikipedia.org/wiki/False_color#True_color).

Oleh karena itu, fungsi rgb() dapat digunakan untuk memvisualisasikan data yang diperoleh dari sebuah query. Fungsi ini memerlukan input minimal sebagai berikut:

* `ds:` Objek xarray.Dataset yang berisi data citra.Dataset` object
* `bands:`Tiga band yang akan ditampilkan (harus ada dalam dataset)
* `index:` Langkah waktu yang akan ditampilkan (default = 0)

### Ploting Satu timestep

Dimensi waktu dalam xarray.Dataset menunjukkan jumlah timesteps yang tersedia untuk lokasi tertentu selama periode yang dipilih.
Dalam fungsi rgb(), variabel index menentukan timestep yang ingin ditampilkan, mirip dengan penggunaan .isel() sebelumnya.
Perlu diperhatikan: Perhitungan indeks dalam Python dimulai dari 0 sehingga untuk menampilkan timestep pertama `index=0` gunakan:

In [ ]:
# View a red, green, blue (true colour) image of the first timestep
rgb(ds, bands=["red", "green", "blue"], index=0)

Dengan mengubah input band, kita dapat memvisualisasikan citra false color, yang dapat memberikan pandangan berbeda tentang suatu lanskap.

Kombinasi band (swir_1, nir, green) ini menekankan:

Vegetasi yang sedang tumbuh dalam warna hijau,
Badan air dalam warna biru gelap.

In [ ]:
# View a swir_1, nir, green (false colour) image of the first timestep
rgb(ds, bands=['swir16', 'nir', 'green'], index=0)

### Memplotting beberapa timestep
Seperti yang telah dibahas dalam contoh citra satu band di atas, memplotting beberapa timestep dalam satu plot dapat berguna (misalnya, untuk membandingkan perubahan dari waktu ke waktu).
Fungsi rgb() memungkinkan kita melakukan ini dengan memberikan daftar beberapa citra yang akan diplot menggunakan index=[X, X, ...].
Sebagai contoh, kita dapat menampilkan citra pertama dan kelima dalam dataset dengan menggunakan index=[0, 4] (ingat bahwa dalam Python, perhitungan indeks dimulai dari 0):

In [ ]:
# View a true colour image for the first and fith timesteps
rgb(ds, bands=['red', 'green', 'blue'], index=[0, 4])

Kita juga dapat menggunakan fungsi rgb() untuk memplot seluruh timestep dalam dataset dengan menggunakan sintaks col="time", seperti yang telah ditunjukkan dalam [contoh single band sebelumnya](#Plotting-multiple-timesteps): 

In [ ]:
# Plot all timesteps in the dataset
rgb(ds, bands=['red', 'green', 'blue'], col="time")

### Kustomisasi tampilan plot
Secara default, fungsi rgb() menghasilkan plot dengan robust=True untuk meningkatkan tampilan citra dengan memangkas 2% piksel tergelap dan tercerah, menggunakan persentil ke-2 dan ke-98 dari data untuk menentukan batas warna.

Jika hasil default ini kurang optimal, rentang warna pada plot dapat dikustomisasi menggunakan parameter percentile_stretch. Parameter ini memungkinkan kita memangkas nilai minimum dan maksimum yang paling ekstrem dalam dataset, sehingga meningkatkan kontras dan tampilan plot.

Sebagai contoh, dengan menetapkan percentile_stretch=[0.05, 0.95], kita akan memangkas 5% piksel tergelap dan tercerah, sehingga rentang warna akan lebih fokus pada 90% nilai yang kurang ekstrem:

In [ ]:
rgb(ds, 
    bands=['red', 'green', 'blue'], 
    index=0, 
    percentile_stretch=[0.05, 0.95])


---

## ℹ️ Info

Jika ada pertanyaan atau komentar bisa mengirimkan email ke piksel@big.go.id

**Lisensi:** Skrip dalam notebook ini dilisensikan berdasarkan [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Data Piksel dilisensikan berdasarkan lisensi [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/).

In [ ]:
from datetime import datetime
print(f"Tanggal update: {datetime.now().strftime('%d %B %Y')}")